# CatBoost Implementation

## 1. Install Dependencies

In [1]:
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [2]:
!pip install numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 31.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.22.4-cp311-cp311-linux_x86_64.whl size=17329434 sha256=79db4f5591923502d9377636540e1cfdd708a09c0d50f6ff4182e3ea1c35a573
  Stored in directory: /root/.cache/pip/wheels/8e/c0/7e/1583fa989ccf57e2059824c8783691f4927f2ce7b77cec9da2
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nx-cugraph-cu12 25.2.0 requires numpy<3.0a0,>=1.23, but you have numpy 1.22.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.2

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [3]:
!pip install mlcroissant

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.4/141.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 19.9 MB/s eta 0:00:00
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15127 sha256=9d0e63f4b7e456bea01024bfceb980697e043226a1162f1cde3dc381e028bd46
  Stored in directory: /root/.cache/pip/wheels/53/cf/51/a4ea10224b7fdb523e18e2033cadf2a8657517d1f95f3f5413
Successfully built jsonpath-rw


## 2. Import Dependencies

In [ ]:
import mlcroissant as mlc
import itertools
import joblib
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
warnings.filterwarnings("ignore")

## 3. Download Dataset

In [ ]:
!curl -L -o ~/beth-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/katehighnam/beth-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.7M  100 39.7M    0     0  40.7M      0 --:--:-- --:--:-- --:--:--  118M


In [ ]:
!unzip /root/beth-dataset.zip

Archive:  /root/beth-dataset.zip
replace labelled_2021may-ip-10-100-1-105-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-105-dns.csv  
replace labelled_2021may-ip-10-100-1-105.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-105.csv  
replace labelled_2021may-ip-10-100-1-186-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-186-dns.csv  
replace labelled_2021may-ip-10-100-1-186.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-186.csv  
replace labelled_2021may-ip-10-100-1-26-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-26-dns.csv  
replace labelled_2021may-ip-10-100-1-26.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-ip-10-100-1-26.csv  
replace labelled_2021may-ip-10-100-1-4-dns.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: labelled_2021may-i

## 4. Merge Datasets

In [ ]:
csv_files = [
    'labelled_testing_data.csv',
    'labelled_training_data.csv',
    'labelled_validation_data.csv'
]
combined_data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_data.to_csv('combined_dataset.csv', index=False)

print(f"Combined dataset shape: {combined_data.shape}")
print(combined_data.head())

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

Combined dataset shape: (1141078, 16)
    timestamp  processId  threadId  parentProcessId  userId  mountNamespace  \
0  129.050634        382       382                1     101      4026532232   
1  129.051238        379       379                1     100      4026532231   
2  129.051434          1         1                0       0      4026531840   
3  129.051481          1         1                0       0      4026531840   
4  129.051522          1         1                0       0      4026531840   

       processName         hostName  eventId           eventName  \
0  systemd-resolve  ip-10-100-1-217       41              socket   
1  systemd-network  ip-10-100-1-217       41              socket   
2          systemd  ip-10-100-1-217     1005  security_file_open   
3          systemd  ip-10-100-1-217      257              openat   
4          systemd  ip-10-100-1-217        5               fstat   

                                      stackAddresses  argsNum  returnValue  \


## 5. Classify Columns

In [ ]:
data = pd.read_csv('combined_dataset.csv')

def auto_classify_columns(data):
    categorical_columns = []
    numerical_columns = []

    for col in data.columns:
        if data[col].dtype == 'object':
            unique_vals = data[col].nunique()
            if unique_vals < 0.1 * len(data):
                categorical_columns.append(col)
            else:
                try:
                    data[col] = pd.to_numeric(data[col], errors='raise')
                    numerical_columns.append(col)
                except:
                    categorical_columns.append(col)
        else:
            numerical_columns.append(col)

    return categorical_columns, numerical_columns

categorical_columns, numerical_columns = auto_classify_columns(data)

print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py:14: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.sparse import csr_matrix, issparse


Categorical columns: ['processName', 'hostName', 'eventName', 'stackAddresses', 'args']
Numerical columns: ['timestamp', 'processId', 'threadId', 'parentProcessId', 'userId', 'mountNamespace', 'eventId', 'argsNum', 'returnValue', 'sus', 'evil']


## 6. Feature Encoding and Scaling

### Encoding with LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

X = data.drop(columns=['evil', 'stackAddresses', 'args', 'argsNum', 'returnValue'])
y = data['evil']

X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

### Scaling with Standard Scaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Shape of X after scaling:", X_scaled.shape)

Shape of X after scaling: (1141078, 11)


## 7. Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

## 8. Train Catboost Model

In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    cat_features=[],
    verbose=100
)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on the test set
y_pred_cat = catboost_model.predict(X_test)
y_prob_cat = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the model using various metrics
accuracy_cat = accuracy_score(y_test, y_pred_cat)
precision_cat = precision_score(y_test, y_pred_cat)
recall_cat = recall_score(y_test, y_pred_cat)
f1_cat = f1_score(y_test, y_pred_cat)
cm_cat = confusion_matrix(y_test, y_pred_cat)

# Print evaluation metrics
print(f"CatBoost Model Evaluation:")
print(f"Accuracy: {accuracy_cat:.4f}")
print(f"Precision: {precision_cat:.4f}")
print(f"Recall: {recall_cat:.4f}")
print(f"F1 Score: {f1_cat:.4f}")


0:	learn: 0.3115333	total: 493ms	remaining: 8m 12s
100:	learn: 0.0000699	total: 28.8s	remaining: 4m 15s
200:	learn: 0.0000634	total: 46.8s	remaining: 3m 6s
300:	learn: 0.0000632	total: 1m 3s	remaining: 2m 27s
400:	learn: 0.0000632	total: 1m 19s	remaining: 1m 58s
500:	learn: 0.0000632	total: 1m 35s	remaining: 1m 35s
600:	learn: 0.0000632	total: 1m 52s	remaining: 1m 14s
700:	learn: 0.0000632	total: 2m 9s	remaining: 55.2s
800:	learn: 0.0000632	total: 2m 27s	remaining: 36.5s
900:	learn: 0.0000632	total: 2m 43s	remaining: 17.9s
999:	learn: 0.0000632	total: 2m 59s	remaining: 0us
CatBoost Model Evaluation:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


### 9. Save Model

In [ ]:
joblib.dump(catboost_model, "catboost_model.pkl")
joblib.dump(y_test, 'y_test_catboost.joblib')
joblib.dump(y_prob_cat, 'y_prob_catboost.joblib')

print("Model and related artifacts saved successfully.")

Model and related artifacts saved successfully.


### 10. Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred_cat)

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
plt.title("Confusion Matrix - CatBoost")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


## 11. ROC Curve

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_prob_cat)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")  # Random classifier line
plt.title("ROC Curve - CatBoost")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

## 12. Precision Recall

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, y_prob_cat)
pr_auc = auc(recall, precision)

plt.figure(figsize=(6,6))
plt.plot(recall, precision, color="blue", lw=2, label=f"PR Curve (AUC = {pr_auc:.2f})")
plt.title("Precision-Recall Curve - CatBoost")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower left")
plt.show()

## 13. Residuals Distribution

In [ ]:
residuals = y_test - y_pred_cat

plt.figure(figsize=(6,4))
sns.histplot(residuals, bins=20, kde=True, color="red")
plt.title("Residuals Distribution - CatBoost")
plt.xlabel("Residuals (y_test - y_pred)")
plt.ylabel("Count")
plt.show()
